In [2]:
import tensorflow as tf
import numpy as np

fields = ["Year", "Month", "Day", "Time", "Global_active_power", "Global_reactive_power", "Voltage", 
          "Global_intensity", "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]

filename_queue = tf.train.string_input_producer(
    ["normalized.csv"], shuffle=False, name='filename_queue')

# set tensorflow reader
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# set record_defaults corresponding to data form
record_defaults = [[0.]]*7  # record_defaults = [[0.], [0.], [0.], [0.]]
data = tf.decode_csv(value, record_defaults=record_defaults)

# set collecting data and batch option
train_x_batch, train_y_batch = tf.train.batch([data[1:], data[:1]], batch_size=64)


In [3]:
# configs for learning
learning_rate = 0.001
num_epochs = 4
batch_size = 128
display_step = 1 # how often print cost?
input_size = 6
hidden1_size = 15
hidden2_size = 15
output_size = 1

# placeholders for input/output
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

In [4]:
# ANN model
def build_ANN(x):
    W1 = tf.Variable(tf.random_normal(shape = [input_size, hidden1_size]))
    b1 = tf.Variable(tf.random_normal(shape = [hidden1_size]))
    H1_output = tf.nn.relu(tf.matmul(x, W1) + b1)
    
    W2 = tf.Variable(tf.random_normal(shape = [hidden1_size, hidden2_size]))
    b2 = tf.Variable(tf.random_normal(shape = [hidden2_size]))
    H2_output = tf.nn.relu(tf.matmul(H1_output, W2)+b2)
    
    W_output = tf.Variable(tf.random_normal(shape = [hidden2_size, output_size]))
    b_output = tf.Variable(tf.random_normal(shape = [output_size]))
    logits = tf.matmul(H2_output, W_output) + b_output
    
    return logits


In [5]:
prediction = build_ANN(x)

# define loss and optimizer
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
loss = tf.reduce_mean(tf.square(prediction - y))
train_step = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# 횟수(epoch)만큼 최적화
for epoch in range(num_epochs):
    average_loss = 0

    total_batch = 10000
    # 2049280
    for i in range(total_batch):
        batch_x, batch_y = sess.run([train_x_batch, train_y_batch])
        #batch_x = np.hstack((x1_batch,x2_batch))
        
        _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
        average_loss += current_loss / total_batch

    if epoch % display_step == 0:
        print("Epoch: " + str(epoch + 1) + ", Loss: " + str(average_loss))

coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, normalized.csv; No such file or directory
	 [[{{node ReaderReadV2}}]]


OutOfRangeError: FIFOQueue '_4_batch_1/fifo_queue' is closed and has insufficient elements (requested 64, current size 0)
	 [[node batch_1 (defined at <ipython-input-2-80ca84028bd7>:19) ]]

Caused by op 'batch_1', defined at:
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in s

In [7]:
print("estimated value: ",sess.run(prediction, feed_dict={x: [[0.212,-0.024,0.285,-0.013,-0.004,0.34]]}))

estimated value:  [[0.28787938]]
